# Nonlinear Glucose Model Without Delay

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gekko import GEKKO
import pandas as pd
from scipy.interpolate import interp1d

In [ ]:
# import data from Parsed.csv
df = pd.read_csv('Parsed.csv')
# df.describe()
df.head(20)

In [ ]:
# # MHE/MPC Model using NlGM
# m = GEKKO()

# m.time = np.linspace(0, 180, 181) # Time horizon of 3 hours

# # FV

# # Const
# Kx = m.Const(0.01)  # (1/min) Rate of insulin dispered from injection site
# nT

# # Vars
# I = m.Var()  # Plasma Insulin
# Q = m.Var()  # Interstitial Insulin Concentration
# Ps = m.Var() # glucose absorbed into the gut
# Us = m.Var() # Subcutaneous glucose concentration
# Px = m.Var() # regular meals of varying glucose content ~450 mmol for each meal
# Pc = m.Var() # Snack Function - three snacks a day of 160 mmol
# ϵ = m.Var()  # exercise level
# SI = m.Var() # insulin sensitivity

# # MV
# Ux = m.MV()  # insulin infusion

# # CV
# G = m.CV()   # Blood Glucose Concentration (mg/dl)

# # Equations

# # Options

# Bergman Model

In [ ]:
# Objective function variables
target = 80 # target
slope = 0.7
ub = 105     # upper bound
lb = 65    # lower bound
lm = 10     # lower penalty multiplier

glucose_ = np.linspace(0, 300, 3000)
reward = []
for g in glucose_:
    if g < lb:
        r = -1 * lm
    elif g > ub:
        r = -1
    else:
        r = 1.0 - np.tanh(np.abs((g - target) / slope) * .1) **2
    reward.append(r)
    
# plot reward function
plt.title('Glucose Reward Function')
plt.plot(glucose_[500:1200], reward[500:1200])
plt.xlabel('Glucose Level')
plt.ylabel('Reward')
plt.show()


reward_func = interp1d(glucose_, reward)

In [ ]:
# For m in [mhe, sim]
m = GEKKO()

# TODO Find optimal control horizon to account for disturbances especially
m.time = np.linspace(0, 180, 181)  # Three hour moving horizon

# Params
p1 = m.Param(3.17e-2)   # (1/min)
p2 = m.Param(1.23e-2)   # (1/min)
si = m.Param(2.9e-2)    # (1/min * (mL/micro-U))
ke = m.Param(9.0e-2)    # (1/min) Insulin elimination from plasma
kabs = m.Param(1.2e-2)  # (1/min) t max,G inverse
kemp = m.Param(1.8e-1)  # (1/min) t max,I inverse
f = m.Param(8.00e-1)    # (L)
vi = m.Param(12.0)      # (L)     Insulin distribution volume
vg = m.Param(12.0)      # (L)     Glucose distibution volume

final = np.zeros_like(m.time)
final[-1] = 1
fin = m.Param(final)

# FV
# TODO implement good bounds on these
# TODO Bounds on possible levels
bg = m.FV(291.0)  # (mg/dL) Basal Blood Glucose
D = m.FV()      # Disturbance (mmol/L min)

# storage arrays
bg_ = np.zeros_like(m.time)
D_  = np.zeros_like(m.time)

# Vars
# TODO: Initial Values
G = m.CV(100)    # (mg/dL) Blood Glucose
X = m.Var()      # (μu/ml) Remote Insulin
I = m.Var()      # (μu/ml) Plasma Insulin
U = m.MV()       # (mU/min) Insulin Delivery
GG = m.Var()     # (mg/dL) Gut insulin

S1 = m.Var()     # Intermediate 1
S2 = m.Var()     # Intermediate 2

obj = m.Var(0)

# Equations
m.Equations([
    G.dt() == -p1 * (G-bg) - si * X * G + f * kabs / vg * GG + f / vg * D,  # Dynamic Insulin Rate
    X.dt() == p2 * (I - X),                                                 # Remote Insulin dynamics
    I.dt() == -ke * I + U,                                                  # Plasma Insulin Concentration
    S1.dt() == U - kemp * S1,                                               # Dynamic Model
    S2.dt() == -kemp * (S2 - S1),                                           # Dynamic Model
    GG.dt() == kemp * S2 - kabs * GG,
    obj.dt() == m.if2(G-lb, -1 * lm, m.if2(ub-g, -1, 1.0 - m.tanh(m.abs((G-target)/slope)*0.1)**2)) # m.integral(print(reward_func(G)[0]))
    
])

# Options
## Tuning
bg.STATUS = 1; bg.FSTATUS = 0
D.STATUS  = 1; D.FSTATUS  = 0

G.STATUS  = 0; G.FSTATUS  = 1
U.STATUS  = 0; U.FSTATUS  = 1

## Global options
m.options.IMODE   = 5
m.options.SOLVER  = 3
m.options.MAX_ITER = 500

m.Maximize(obj * fin)

In [ ]:
# iterate through rows and update with MPC
# TODO Can we use measurements every 5 minutes for one variable and every 5 for another?
i=0
for date, time, glucose, insulin_delivery in df.itertuples(index=False):
    print(f'iteration {i}'.center(30,'-'))
    # only update glucose measurement if not NaN
    if not np.isnan(glucose):
        G.MEAS = glucose
    
    U.MEAS = insulin_delivery
    print(f'{G.value=}')
    print(f'{U.value=}')
    # solve with MHE
    m.solve(disp=False)
    
    # Store MHE values for basal glucose rate and disturbances
    print(f'{bg.NEWVAL=}')
    print(f'{D.NEWVAL=}')
    bg_[i] = bg.NEWVAL
    D_[i]  = D.NEWVAL
    
    i += 1
        
    

In [ ]:
# TODO analyze inputs and outputs to create ARX model

In [ ]:
# Control Based on ARX model & simulated disturbances
def get_situation():
    '''return parameters neccesary to simulate a day'''
    # Meal data for disturbances
    # order: [breakfast, snack1, lunch, snack2, dinner, snack3]
    prob_meal = 
    return

def sim_day():
    '''Simulate a day'''
    return
